In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
# 예시 GPU 데이터
data = {
    'GPU Model': ['NVIDIA RTX 3080', 'AMD RX 6800 XT', 'NVIDIA GTX 1660', 'AMD RX 570'],
    'Cores': [8704, 4608, 1408, 2048],
    'VRAM': [10, 16, 6, 4],
    'Memory Speed': [19, 16, 8, 7],
    'TDP': [320, 300, 120, 150],
    'Price': [699, 649, 219, 179],
    'Manufacturer': ['NVIDIA', 'AMD', 'NVIDIA', 'AMD'],
    'Core Clock': [1.44, 1.82, 1.53, 1.168],
    'Boost Clock': [1.71, 2.25, 1.785, 1.244],
    'Ray Tracing': [1, 1, 0, 0]
}

gpu_data = pd.DataFrame(data)
gpu_data.to_csv('gpu_data.csv', index=False)

In [ ]:
user_purpose = '게임용'
user_budget = 700

# 용도별 사전 정의된 프로필
profiles = {
    '게임용': {
        'Cores': 8000,
        'VRAM': 8,
        'Memory Speed': 14,
        'TDP': 300,
        'Price': user_budget,
        'Core Clock': 1.5,
        'Boost Clock': 1.8,
        'Ray Tracing': 1  # 1: Yes, 0: No
    },
    '사무용': {
        'Cores': 1500,
        'VRAM': 4,
        'Memory Speed': 10,
        'TDP': 75,
        'Price': user_budget,
        'Core Clock': 1.2,
        'Boost Clock': 1.4,
        'Ray Tracing': 0
    },
    '영상편집': {
        'Cores': 4000,
        'VRAM': 12,
        'Memory Speed': 16,
        'TDP': 250,
        'Price': user_budget,
        'Core Clock': 1.6,
        'Boost Clock': 2.0,
        'Ray Tracing': 1
    }
}

# 입력된 용도에 따라 프로필 선택
user_profile = profiles.get(user_purpose, profiles['사무용'])  # 기본값은 '사무용'

# 3. 유사도 계산
def compute_similarity(gpu_data, user_profile):
    features = ['Cores', 'VRAM', 'Memory Speed', 'TDP', 'Price', 'Core Clock', 'Boost Clock', 'Ray Tracing']
    user_vector = np.array([user_profile[feature] for feature in features]).reshape(1, -1)
    gpu_vectors = gpu_data[features].values
    similarities = cosine_similarity(user_vector, gpu_vectors)
    gpu_data['Similarity'] = similarities[0]
    return gpu_data

gpu_data = compute_similarity(gpu_data, user_profile)

# 4. 추천 생성
recommendations = gpu_data[gpu_data['Price'] <= user_profile['Price']].sort_values(by='Similarity', ascending=False)

print("추천 GPU 리스트:")
print(recommendations[['GPU Model', 'Similarity']].head())

추천 GPU 리스트:
         GPU Model  Similarity
0  NVIDIA RTX 3080    0.999974
3       AMD RX 570    0.999368
1   AMD RX 6800 XT    0.998249
2  NVIDIA GTX 1660    0.996662


In [ ]:
# 예시 GPU 데이터
data = {
    'GPU Model': ['NVIDIA RTX 3080', 'AMD RX 6800 XT', 'NVIDIA GTX 1660', 'AMD RX 570'],
    'Cores': [8704, 4608, 1408, 2048],
    'VRAM': [10, 16, 6, 4],
    'Memory Speed': [19, 16, 8, 7],
    'TDP': [320, 300, 120, 150],
    'Price': [699, 649, 219, 179],
    'Manufacturer': ['NVIDIA', 'AMD', 'NVIDIA', 'AMD'],
    'Core Clock': [1.44, 1.82, 1.53, 1.168],
    'Boost Clock': [1.71, 2.25, 1.785, 1.244],
    'Ray Tracing': [1, 1, 0, 0]
}

gpu_data = pd.DataFrame(data)
gpu_data.to_csv('gpu_data.csv', index=False)

user_purpose = '게임용'
user_budget = 700

# 용도별 사전 정의된 프로필
profiles = {
    '게임용': {
        'Cores': 8000,
        'VRAM': 8,
        'Memory Speed': 14,
        'TDP': 300,
        'Price': user_budget,
        'Core Clock': 1.5,
        'Boost Clock': 1.8,
        'Ray Tracing': 1  # 1: Yes, 0: No
    },
    '사무용': {
        'Cores': 1500,
        'VRAM': 4,
        'Memory Speed': 10,
        'TDP': 75,
        'Price': user_budget,
        'Core Clock': 1.2,
        'Boost Clock': 1.4,
        'Ray Tracing': 0
    },
    '영상편집': {
        'Cores': 4000,
        'VRAM': 12,
        'Memory Speed': 16,
        'TDP': 250,
        'Price': user_budget,
        'Core Clock': 1.6,
        'Boost Clock': 2.0,
        'Ray Tracing': 1
    }
}

# 입력된 용도에 따라 프로필 선택
user_profile = profiles.get(user_purpose, profiles['사무용'])  # 기본값은 '사무용'

# 3. 유사도 계산
def compute_similarity(gpu_data, user_profile):
    features = ['Cores', 'VRAM', 'Memory Speed', 'TDP', 'Price', 'Core Clock', 'Boost Clock', 'Ray Tracing']
    user_vector = np.array([user_profile[feature] for feature in features]).reshape(1, -1)
    gpu_vectors = gpu_data[features].values
    similarities = cosine_similarity(user_vector, gpu_vectors)
    gpu_data['Similarity'] = similarities[0]
    return gpu_data

gpu_data = compute_similarity(gpu_data, user_profile)

# 4. 추천 생성
recommendations = gpu_data[gpu_data['Price'] <= user_profile['Price']].sort_values(by='Similarity', ascending=False)

print("추천 GPU 리스트:")
print(recommendations[['GPU Model', 'Similarity']].head())

추천 GPU 리스트:
         GPU Model  Similarity
0  NVIDIA RTX 3080    0.999974
3       AMD RX 570    0.999368
1   AMD RX 6800 XT    0.998249
2  NVIDIA GTX 1660    0.996662


In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 예시: 사용자로부터 용량, DDR 유형, 예산 입력받기
user_capacity = int(input("RAM 용량을 입력하세요 (GB 단위): "))
user_ddr = int(input("DDR 유형을 입력하세요 (예: 3, 4, 5): "))
user_budget = int(input("예산을 입력하세요 (달러 단위): "))

# 사용자 프로필 생성
user_profile = {
    'Capacity': user_capacity,
    'DDR': user_ddr,
    'Price': user_budget
}

# 예시 RAM 데이터
data = {
    'RAM Model': ['Corsair Vengeance LPX 16GB', 'G.SKILL Ripjaws V 32GB', 'Kingston HyperX Fury 8GB', 'Crucial Ballistix 16GB'],
    'Capacity': [16, 32, 8, 16],
    'DDR': [4, 4, 3, 3],
    'Price': [80, 160, 40, 70],
    'Manufacturer': ['Corsair', 'G.SKILL', 'Kingston', 'Crucial']
}

ram_data = pd.DataFrame(data)
ram_data.to_csv('ram_data.csv', index=False)

# 3. 유사도 계산
def compute_similarity(ram_data, user_profile):
    features = ['Capacity', 'DDR', 'Price']
    user_vector = np.array([user_profile[feature] for feature in features]).reshape(1, -1)
    ram_vectors = ram_data[features].values
    similarities = cosine_similarity(user_vector, ram_vectors)
    ram_data['Similarity'] = similarities[0]
    return ram_data

ram_data = compute_similarity(ram_data, user_profile)

# 4. 추천 생성
recommendations = ram_data[ram_data['Price'] <= user_profile['Price']].sort_values(by='Similarity', ascending=False)

print("추천 RAM 리스트:")
print(recommendations[['RAM Model', 'Similarity']].head())


RAM 용량을 입력하세요 (GB 단위): 16
DDR 유형을 입력하세요 (예: 3, 4, 5): 3
예산을 입력하세요 (달러 단위): 200
추천 RAM 리스트:
                    RAM Model  Similarity
1      G.SKILL Ripjaws V 32GB    0.993054
0  Corsair Vengeance LPX 16GB    0.992527
2    Kingston HyperX Fury 8GB    0.991408
3      Crucial Ballistix 16GB    0.989174


In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 예시: 사용자로부터 CPU 세대, M.2 지원 여부, DDR 유형 입력받기
user_cpu_generation = int(input("CPU 세대를 입력하세요: "))
user_m2_support = int(input("M.2 지원 여부를 입력하세요 (지원: 1, 미지원: 0): "))
user_ddr = int(input("DDR 유형을 입력하세요 (예: 3, 4, 5): "))

# 사용자 프로필 생성
user_profile = {
    'CPU Generation': user_cpu_generation,
    'M.2 Support': user_m2_support,
    'DDR': user_ddr
}

# 예시 메인보드 데이터
data = {
    'Mainboard Model': ['ASUS ROG Strix Z490-E', 'MSI MPG Z390 Gaming Pro', 'Gigabyte B450 AORUS M', 'ASRock B460M Pro4'],
    'CPU Generation': [10, 9, 3, 10],
    'M.2 Support': [1, 1, 1, 0],
    'DDR': [4, 4, 4, 4],
    'Price': [250, 180, 100, 90],
    'Manufacturer': ['ASUS', 'MSI', 'Gigabyte', 'ASRock']
}

mainboard_data = pd.DataFrame(data)
mainboard_data.to_csv('mainboard_data.csv', index=False)

# 3. 유사도 계산
def compute_similarity(mainboard_data, user_profile):
    features = ['CPU Generation', 'M.2 Support', 'DDR']
    user_vector = np.array([user_profile[feature] for feature in features]).reshape(1, -1)
    mainboard_vectors = mainboard_data[features].values
    similarities = cosine_similarity(user_vector, mainboard_vectors)
    mainboard_data['Similarity'] = similarities[0]
    return mainboard_data

mainboard_data = compute_similarity(mainboard_data, user_profile)

# 4. 추천 생성
recommendations = mainboard_data.sort_values(by='Similarity', ascending=False)

print("추천 메인보드 리스트:")
print(recommendations[['Mainboard Model', 'Similarity']].head())

CPU 세대를 입력하세요: 10
M.2 지원 여부를 입력하세요 (지원: 1, 미지원: 0): 1
DDR 유형을 입력하세요 (예: 3, 4, 5): 3
추천 메인보드 리스트:
           Mainboard Model  Similarity
0    ASUS ROG Strix Z490-E    0.996068
1  MSI MPG Z390 Gaming Pro    0.992037
3        ASRock B460M Pro4    0.991500
2    Gigabyte B450 AORUS M    0.804054


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 예시 데이터
data = {
    'Case Model': ['NZXT H510', 'Cooler Master Q300L', 'Fractal Design Meshify C', 'Corsair 275R Airflow'],
    'Price': [100, 50, 90, 80],
    'CPU Form Factor': ['ATX', 'Micro-ATX', 'ATX', 'ATX']
}

case_data = pd.DataFrame(data)

# 예시: 사용자로부터 가격 및 CPU 폼팩터 입력받기
user_price = float(input("가격을 입력하세요: "))
user_cpu_form_factor = input("CPU 폼팩터를 입력하세요 (예: ATX, Micro-ATX): ")

# 사용자 프로필 생성
user_profile = {
    'Price': user_price,
    'CPU Form Factor': user_cpu_form_factor
}

# 1. 가격 표준화
scaler = StandardScaler()
case_data['Normalized Price'] = scaler.fit_transform(case_data[['Price']])

# 2. CPU 폼팩터 원핫 인코딩
case_data = pd.get_dummies(case_data, columns=['CPU Form Factor'])

# 3. 사용자 프로필 데이터 변환
user_profile['Normalized Price'] = scaler.transform([[user_profile['Price']]])[0][0]
user_profile_vector = [user_profile['Normalized Price']] + [0] * (len(case_data.columns) - 3)

if 'CPU Form Factor_' + user_profile['CPU Form Factor'] in case_data.columns:
    index = case_data.columns.get_loc('CPU Form Factor_' + user_profile['CPU Form Factor'])
    user_profile_vector[index - 1] = 1  # Adjusted index for user vector

user_vector = np.array(user_profile_vector).reshape(1, -1)
case_vectors = case_data.drop(columns=['Case Model', 'Price']).values

# 4. 유사도 계산
similarities = cosine_similarity(user_vector, case_vectors)
case_data['Similarity'] = similarities[0]

# 5. 추천 생성
recommendations = case_data.sort_values(by='Similarity', ascending=False)

print("추천 케이스 리스트:")
print(recommendations[['Case Model', 'Similarity']])

가격을 입력하세요: 100
CPU 폼팩터를 입력하세요 (예: ATX, Micro-ATX): ATX
추천 케이스 리스트:
                 Case Model  Similarity
0                 NZXT H510    0.533333
2  Fractal Design Meshify C    0.344265
3      Corsair 275R Airflow    0.000000
1       Cooler Master Q300L   -0.258199


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 예시 데이터
data = {
    'Power Supply Model': ['Corsair RM750x', 'EVGA SuperNOVA 750 G5', 'Seasonic FOCUS GX-650', 'Cooler Master MWE Gold 650 V2'],
    'Price': [120, 110, 90, 80],
    'ATX Support': ['Yes', 'Yes', 'Yes', 'No'],
    'CPU Type': ['External', 'External', 'External', 'Internal']
}

psu_data = pd.DataFrame(data)

# 사용자 입력
user_price = 100
user_atx_support = 'Yes'
user_cpu_type = 'External'

# 사용자 프로필 생성
user_profile = {
    'Price': user_price,
    'ATX Support': user_atx_support,
    'CPU Type': user_cpu_type
}

# 1. 가격 표준화
scaler = StandardScaler()
psu_data['Normalized Price'] = scaler.fit_transform(psu_data[['Price']])

# 2. ATX 지원 여부 및 CPU 타입 원핫 인코딩
psu_data = pd.get_dummies(psu_data, columns=['ATX Support', 'CPU Type'])

# 3. 사용자 프로필 데이터 변환
user_profile['Normalized Price'] = scaler.transform([[user_profile['Price']]])[0][0]

# 유저 프로필 벡터 크기 설정 (열 이름에 따라)
user_profile_vector = [user_profile['Normalized Price']]

# 원핫 인코딩된 열에 맞추어 사용자 프로필 벡터 업데이트
for column in psu_data.columns[2:]:  # 'Power Supply Model'과 'Price'를 제외한 모든 열
    if column == 'Normalized Price':
        continue
    if column == 'ATX Support_' + user_profile['ATX Support'] or column == 'CPU Type_' + user_profile['CPU Type']:
        user_profile_vector.append(1)
    else:
        user_profile_vector.append(0)

user_vector = np.array(user_profile_vector).reshape(1, -1)
psu_vectors = psu_data.drop(columns=['Power Supply Model', 'Price']).values

# 4. 유사도 계산
similarities = cosine_similarity(user_vector, psu_vectors)
psu_data['Similarity'] = similarities[0]

# 5. 추천 생성
recommendations = psu_data.sort_values(by='Similarity', ascending=False)

print("추천 파워 서플라이 리스트:")
print(recommendations[['Power Supply Model', 'Similarity']])


추천 파워 서플라이 리스트:
              Power Supply Model  Similarity
1          EVGA SuperNOVA 750 G5    0.912871
2          Seasonic FOCUS GX-650    0.912871
0                 Corsair RM750x    0.745356
3  Cooler Master MWE Gold 650 V2    0.000000


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 예시 SSD 데이터
ssd_data = {
    'SSD Model': ['Samsung 970 EVO', 'Western Digital WD Black', 'Crucial MX500', 'Samsung 860 EVO'],
    'Capacity (GB)': [500, 1000, 500, 1000],
    'M.2': ['Yes', 'Yes', 'No', 'No'],
    'Price': [150, 200, 100, 120]
}

ssd_df = pd.DataFrame(ssd_data)

# 사용자 입력
user_capacity = 500  # GB
user_m2 = 'Yes'

# 사용자 프로필 생성
user_profile = {
    'Capacity (GB)': user_capacity,
    'M.2': user_m2
}

# 1. 가격 표준화
scaler = StandardScaler()
ssd_df['Normalized Price'] = scaler.fit_transform(ssd_df[['Price']])

# 2. M.2 지원 여부 원핫 인코딩
ssd_df = pd.get_dummies(ssd_df, columns=['M.2'])

# 3. 사용자 프로필 데이터 변환
user_profile['Normalized Price'] = scaler.transform([[user_profile['Capacity (GB)']]])[0][0]

# 4. 사용자 프로필 벡터 생성
user_profile_vector = [user_profile['Capacity (GB)'], user_profile['Normalized Price']]

# 원핫 인코딩된 열에 맞추어 사용자 프로필 벡터 업데이트
m2_columns = [col for col in ssd_df.columns if col.startswith('M.2')]
for column in m2_columns:
    if column == 'M.2_' + user_profile['M.2']:
        user_profile_vector.append(1)
    else:
        user_profile_vector.append(0)

user_vector = np.array(user_profile_vector).reshape(1, -1)

# 5. SSD 벡터 생성
ssd_vectors = ssd_df.drop(columns=['SSD Model', 'Price']).values

# 6. 유사도 계산
similarities = cosine_similarity(user_vector, ssd_vectors)
ssd_df['Similarity'] = similarities[0]

# 7. 추천 생성
recommendations = ssd_df.sort_values(by='Similarity', ascending=False)

print("추천 SSD 리스트:")
print(recommendations[['SSD Model', 'Similarity']])


추천 SSD 리스트:
                  SSD Model  Similarity
1  Western Digital WD Black    0.999847
0           Samsung 970 EVO    0.999827
3           Samsung 860 EVO    0.999806
2             Crucial MX500    0.999771


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
